# Workflow Demonstration



In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# Add this directory to the system path
sys.path.append(parent_dir)

import src.pipeline.data_preparation_pipeline as data_pipe
import src.pipeline.recommender_pipeline as recommender_pipe
import pandas as pd

import src.models.mapper.skill_mapper as skill_mapper
import src.models.transformer.occ_transformer as occ_transformer
import src.models.occ_mapper.occ_mapper as occ_mapper
import src.models.matcher.matcher as matcher
import src.models.recommender.recommender as recommender
import src.data.renaming_helper as renaming_helper
import src.models.matcher.matcher as matcher
import src.data.esco_helper as EscoHelper

## Data Preparation
Here, we set up paths to data files and initialize various components like the SkillMapper, OccFSMapper, and OccTransformer. The mapper is used to map skills, while the transformer converts user profiles based on their reference data. We also load user profiles into a DataFrame and prepare them for further processing. When using the framework this part ist convered within the source files of the api folder. 

In [2]:

data_path = "../data/"
esco_path = "../data/ESCO/"
path_to_mapping = data_path + "FutureSkills/FutureSkillMappings/processed/MappingTable_test_new.csv"


mapper = skill_mapper.SkillMapper(path_to_mapping)

# initialize occFSmapper with file from OccupationFSProfiles vanilla
occ_fs_mapper = occ_mapper.OccFSMapper()

# initialize transformer with reference user profiles
reference_user_profiles = "../data/UserData/reference_user_profiles_20240528T143048Z.csv"
# reference_user_profiles = r'path_to\reference_user_profiles.csv'

transformer = occ_transformer.OccTransformer(reference_user_profiles=reference_user_profiles)
esco_helper = EscoHelper.esco_helper(esco_path)

# import user profiles to get profile of one user
user_profiles = pd.read_csv(reference_user_profiles, index_col=0)
ratings = renaming_helper.rename_columns(user_profiles.iloc[:, 8:18])
job_matcher = matcher.JobMatcher()
job_recommender = recommender.JobRecommender(ratings.iloc[1].values)


## Running the Data Preparation Pipeline
In this part, we instantiate and run the data preparation pipeline. The pipeline processes the user profiles, transforming them using the mappers and transformers initialized earlier. The output, transformed_profiles, is the processed data ready for use in the recommendation system.

In [3]:
# data preparation pipeline
datapipe = data_pipe.DataPrepPipeline(data_path=data_path,mapper=mapper, occ_mapper=occ_fs_mapper,
                         transformer=transformer)
transformed_profiles = datapipe.run()

MappingTable_test_new_k=10_20240528T143048Z.csv
is already available!


## Preparing an Example User Profile, Renaming Columns and Adding Job History
This section selects an example user profile based on the index and prepares it for the recommendation process. The selected profile will be the target for which job recommendations are generated. To ensure consistency, we rename the columns in both the user profiles and the target profile to match the format used in the transformed profiles. Furthermore, we add a job history example for testing. This data simulates the user's past job experiences, which can influence the job recommendation process.

In [13]:
target_profile = user_profiles.loc[46].to_frame().T
job_recommendations = "http://data.europa.eu/esco/occupation/4163b31e-440b-4991-b5c3-48d0515250ef,http://data.europa.eu/esco/occupation/4a3f40a8-0587-494c-b8d3-7098b8c5992f,http://data.europa.eu/esco/occupation/8f9b20ff-86b3-478c-8054-73a220973d6f,http://data.europa.eu/esco/occupation/c2efadc4-6f47-4f34-997b-dd80aba609dd"
job_ratings = "0,0,0,-1"
target_profile['job_recommendations'] = job_recommendations
target_profile['job_ratings'] = job_ratings


In [14]:
# rename user_profile und target_profile columns to match the transformed_profiles columns before passing to recommender pipeline
column_map = {
    'self_initiative': 'FS1',
    'flexibility': 'FS2',
    'leadership': 'FS3',
    'communication': 'FS4',
    'creativity': 'FS5',
    'customer_orientation': 'FS6',
    'organization': 'FS7',
    'problem_solving': 'FS8',
    'resilience': 'FS9',
    'goal_orientation': 'FS10'
}

user_profiles.rename(columns=column_map, inplace=True)
target_profile.rename(columns=column_map, inplace=True)
transformed_profiles.rename_axis('conceptUri', inplace=True)

In [15]:
# Add job_history example for testing

job_history = [
    {"http://data.europa.eu/esco/occupation/258e46f9-0075-4a2e-adae-1ff0477e0f30": True},
    {"http://data.europa.eu/esco/occupation/2f7f5c84-b9b3-41ea-aa5f-5997d0873141": False}
]

## Running the Recommender Pipeline
Finally, we set up and run the recommender pipeline. The pipeline takes the prepared profiles, user preferences, and job history as input and generates job recommendations. The recommended job is then looked up using the EscoHelper to retrieve the job label, which is printed as the final output.

In [16]:
# recommender pipeline
recom_pipe = recommender_pipe.RecommenderPipeline(data_path=data_path, matcher=job_matcher, recommender=job_recommender)

recommendation = recom_pipe.run(target_profile=target_profile, preferences=[1, 4, 5, 6], transformed_profiles=transformed_profiles, user_profiles=user_profiles, job_history=job_history)
print('Jobvorschlag: ', esco_helper.look_up_label(recommendation["conceptUri"][0]))


found 1 user profiles that rated the job the same and share preferences
found 0 user profiles that rated the job the same but dont share preferences
Jobvorschlag:  Regisseur/Regisseurin


c:\Users\SAE\Desktop\VS_Code\KITA\kira-webscraping\src\models\recommender\recommender.py:127: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_recom = pd.concat([recommended_jobs, recommended_jobs_broader_occ], axis=0, ignore_index=False)
